In [2]:
#https://influxdb-python.readthedocs.io/en/latest/api-documentation.htm
import pandas as pd
# using Http
# InfluxDB connections settings
import influxdbsettings
host=influxdbsettings.host
port=influxdbsettings.port
username = influxdbsettings.username
password = influxdbsettings.password
database = influxdbsettings.database


In [6]:
''' ## allgemeine infos aus influx auslesen
from influxdb import InfluxDBClient
client = InfluxDBClient(host, port, username, password, database)
client.ping()
users = client.get_list_users()
users
databases= client.get_list_database()
databases
client.get_list_privileges(username)
client.switch_database(database)
measurements=client.get_list_measurements()
measurements
queries= client.get_list_continuous_queries()
queries'''

' ## allgemeine infos aus influx auslesen\nfrom influxdb import InfluxDBClient\nclient = InfluxDBClient(host, port, username, password, database)\nclient.ping()\nusers = client.get_list_users()\nusers\ndatabases= client.get_list_database()\ndatabases\nclient.get_list_privileges(username)\nclient.switch_database(database)\nmeasurements=client.get_list_measurements()\nmeasurements\nqueries= client.get_list_continuous_queries()\nqueries'

In [7]:
# load local mfm-measurements
from influxdb import DataFrameClient
clientDF = DataFrameClient(host,port,username,password,database="telegraf")
influxquery='SELECT "MFM_FREQUENCY_actual" FROM "telegraf"."autogen"."mqtt_consumer" WHERE time > now() - 7d'
dffreq=clientDF.query(influxquery)
dffreq=dffreq['mqtt_consumer']


In [8]:
dffreq=dffreq[dffreq['MFM_FREQUENCY_actual']>40]

In [74]:
#dfambient=clientDF.query('SELECT "DS18B20-1_Temperature", "DS18B20-2_Temperature", "BME280_Temperature", "BME280_Pressure", "BME280_Humidity", "BME280_DewPoint" FROM "telegraf"."autogen"."mqtt_consumer" WHERE time > now() - 30d AND "topic"=\'tele/ESP01/SENSOR\'')
#dfambient=dfambient['mqtt_consumer']
#dfambient.head(2)

,DS18B20-1_Temperature,DS18B20-2_Temperature,BME280_Temperature,BME280_Pressure,BME280_Humidity,BME280_DewPoint
2022-01-06 17:29:53.519554+00:00,1.1,3.6,4.2,1011.6,100.0,4.2
2022-01-06 17:34:53.492321+00:00,1.0,3.5,4.2,1011.5,100.0,4.2


In [9]:
#dfplot=dffreq[dffreq["datetime"]>'2022-01-01']
dfplot=dffreq["2022-01-01":"2022-02-01"]
dfplot=dfplot.resample("min").min()
dfplot['datetime']=dfplot.index
dfplot['day'] = dfplot['datetime'].dt.day
dfplot['dayofweek'] = dfplot['datetime'].dt.day_name()#.dt.dayofweek
dfplot['hour'] = dfplot['datetime'].dt.hour
dfplot['date'] = dfplot['datetime'].dt.date
dfplot['minute'] = dfplot['datetime'].dt.minute
dfplot['year'] = dfplot['datetime'].dt.year
dfplot['month'] = dfplot['datetime'].dt.month

dfplot.head(2)

,MFM_FREQUENCY_actual,datetime,day,dayofweek,hour,date,minute,year,month
2022-01-29 20:06:00+00:00,49.964,2022-01-29 20:06:00+00:00,29,Saturday,20,2022-01-29,6,2022,1
2022-01-29 20:07:00+00:00,49.960,2022-01-29 20:07:00+00:00,29,Saturday,20,2022-01-29,7,2022,1


In [19]:
#dfplot=df.resample("D").mean()
dfplot = dfplot.sort_values(by="hour")
fig = px.line(dfplot, x="hour", y="MFM_FREQUENCY_actual", color='dayofweek', markers=True,title='data from influxDB')
fig.show()

In [116]:
dfplot = dfplot.sort_values(by="hour")
fig = px.scatter(dfplot, x="hour",y="MFM_FREQUENCY_actual", color='MFM_FREQUENCY_actual', title='data from influxDB') #markers=True,
fig.show()

In [13]:
dfplot=dffreq["2022-01-01":"2022-02-01"]
dfplot=dfplot.resample("h").min()
dfplot['datetime']=dfplot.index
dfplot['day'] = dfplot['datetime'].dt.day
dfplot['dayofweek'] = dfplot['datetime'].dt.day_name()#.dt.dayofweek
dfplot['hour'] = dfplot['datetime'].dt.hour
dfplot['date'] = dfplot['datetime'].dt.date
dfplot['minute'] = dfplot['datetime'].dt.minute
dfplot['year'] = dfplot['datetime'].dt.year
dfplot['month'] = dfplot['datetime'].dt.month
dfplot = dfplot.sort_values(by="hour")
fig = px.scatter_3d(dfplot, x="day",y="hour", z="MFM_FREQUENCY_actual",
                    color='MFM_FREQUENCY_actual',
                    symbol='dayofweek',
                    size_max=5,
                    title='data from influxDB') #markers=True,
fig.show()

In [12]:
import plotly.graph_objects as go # or 
import plotly.express as px
fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )
dfplot = dfplot.sort_values(by="hour")
fig = px.scatter_3d(dfplot, x="date",y="hour", z="MFM_FREQUENCY_actual",
                    color='MFM_FREQUENCY_actual',
                    symbol='dayofweek',
                    size_max=5,
                    title='MFM_FREQUENCY_actual') #markers=True,

import dash
from dash import dcc
from dash import html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


## Gridradar API

In [11]:
from influxdb import DataFrameClient
clientDF = DataFrameClient(host,port,username,password,database)

In [15]:
dffreq=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."median_frequency"')
dffreq=dffreq['median_frequency']#[0]
dffreq=dffreq.rename(columns={'value': 'median_frequency'})
dffreq['datetime']=dffreq.index
dffreq=dffreq[dffreq["datetime"]>'2022-01-01']
dffreq.head(2)



,median_frequency,datetime
2022-02-01 21:04:30+00:00,49.93433,2022-02-01 21:04:30+00:00
2022-02-01 21:04:31+00:00,49.93333,2022-02-01 21:04:31+00:00


In [16]:
dftime=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."net_time"')
dftime=dftime['net_time']
dftime=dftime.rename(columns={'value': 'net_time'})

dftime['datetime']=dftime.index
print(dftime["datetime"].max() - dftime["datetime"].min())
dftime.head(2)

139 days 19:11:59


,net_time,datetime
2021-09-19 01:14:02+00:00,5.649146,2021-09-19 01:14:02+00:00
2021-09-19 01:14:03+00:00,5.649344,2021-09-19 01:14:03+00:00


In [18]:
import plotly.express as px
dfplottime=dftime[dftime["datetime"]>'2022-02-02']
fig = px.line(dfplottime,x="datetime",y="net_time", title='gridradar net_time data from influxDB')
fig.show()